# Bodø | Preparation of the municipal tree database 
- author: Willeke A'Campo.
- description: This scripts prepares the raw municipal tree data for input for analysis in accordance with the i-Tree Eco data scheme.
- project: Trekroner 

Tasks of the script:
1. Create the database **bodo_in_situ_trees.gdb** using an XML schema (MANUAL)
    - gdb_path: ```urban-treeDetection/data/%municipality%/interim/bodo_in_situ_trees.gdb```
    - XML schema: ```data/municipality_in_situ_trees.gdb```
2. Copy the  raw tree inventory data into ```bodo_in_situ_trees.gdb``` (MANUAL)
3. Append the raw data into the empty layer ```stem_in_situ``` (MANUAL)
    - use fieldmapping to rename and delete fields 
4. Classify Tree Taxon Data 
    - Reclassify using a look up dictionairy 
    - Look up: ```data/look_up_tables/takson_look_up.xlsx ```

### Set-up

In [ ]:

# import  py-packages 
import os
import logging 
import arcpy 
import pandas as pd

# import local packages
from src import setup_logger 

In [ ]:
# configure logger
setup_logger(logfile=False)

logging.info('Info message')


In [ ]:
#configure log file
log_file = os.path.join(log_path, kommune + "_prepare_tree_db" + ".log"  )
logger = configure_logger(log_file=log_file, set_level= "INFO")
logger.info(f"Municipality is set to '{kommune}'.")

In [ ]:
# print global project variables
print(f'raw_data_path: {raw_data_path}')
print(f'interim_data_path: {interim_data_path}')
print(f'processed_data_path: {processed_data_path}')
print(f'admin_data_path: {admin_data_path}')

In [ ]:
in_situ_trees = os.path.join(interim_data_path, kommune + "_in_situ_trees.gdb")
print(f'in_situ_trees: {in_situ_trees}')

In [ ]:
## APPEND 
arcpy.management.Append(
    inputs="ByTreBodø_lokal",
    target="treetop_in_situ",
    schema_type="NO_TEST",
    field_mapping='tree_id "tree_id" true true false 4 Long 0 0,First,#,ByTreBodø_lokal,OBJECTID,-1,-1;itree_species_code "itree_species_code" true true false 255 Text 0 0,First,#;norwegian_name "norwegian_name" true true false 255 Text 0 0,First,#,ByTreBodø_lokal,Treslag,0,25;taxon_name "taxon_name" true true false 255 Text 0 0,First,#;taxon_type "taxon_type" true true false 255 Text 0 0,First,#;english_name "english_name" true true false 255 Text 0 0,First,#;registration_date "registration_date" true true false 8 Date 0 0,First,#,ByTreBodø_lokal,Registreringsdato,-1,-1;address "address" true true false 255 Text 0 0,First,#,ByTreBodø_lokal,Sted,0,50;land_use "land_use" true true false 255 Text 0 0,First,#;dbh "dbh" true true false 0 Long 0 0,First,#,ByTreBodø_lokal,Omkrets,-1,-1;dbh_heigth "dbh_heigth" true true false 0 Long 0 0,First,#;dieback "dieback" true true false 2 Short 0 0,First,#;crown_missing "crown_missing" true true false 2 Short 0 0,First,#;crown_ligth_exposure "crown_ligth_exposure" true true false 2 Short 0 0,First,#;total_tree_heigth "total_tree_heigth" true true false 4 Float 0 0,First,#;live_tree_heigth "live_tree_heigth" true true false 4 Float 0 0,First,#;crown_base_heigth "crown_base_heigth" true true false 4 Float 0 0,First,#;lat "lat" true true false 8 Double 0 0,First,#;long "long" true true false 8 Double 0 0,First,#',
    subtype="",
    expression="OBJECTID IS NOT NULL",
    match_fields=None,
    update_geometry="NOT_UPDATE_GEOMETRY"
)

### Import Data 

In [ ]:
# load the Excel file into a lookup dictionairy 
excel_path = os.path.join(interim_data_path, 'lookUp.xlsx')
df = pd.read_excel(excel_path)
display(df.head())

lookup_dict= {}
for index, row in df.iterrows():
    #lookup_dict[row['Naturtype']] = row['taksonNorskNavn']
    lookup_dict[row['taksonNorskNavn']] = row['taksonNorskNavn_2']

In [ ]:
# set workspace and shapefile path
arcpy.env.workspace = tree_data_path
fc = "registert_tre"

# Set local variables
inFeatures = fc
field_type = "TEXT"
fieldName = ["taksonNorskNavn","taksonVitNavn", "taksonType","taksonEngelskNavn"]

for name in fieldName:
    # Run AddField twice for two new fields
    arcpy.management.AddField(inFeatures, name, field_type,name, field_is_nullable="NULLABLE")



In [ ]:
# set workspace and shapefile path
arcpy.env.workspace = tree_data_path
fc = "registert_tre"



# specify the field to be modified
field_to_modify = "taksonNorskNavn"

# create an update cursor to modify the values in the field
with arcpy.da.UpdateCursor(fc, [field_to_modify]) as cursor:
    for row in cursor:
        # check the current value of the field and update it to the value from the lookup table
        if row[0] in lookup_dict:
            print(f"Reclassify value <<{row[0]}>> to value <<{lookup_dict[row[0]]}>>..") 
            row[0] = lookup_dict[row[0]]
        #if row[0] is None:
            #row[0] = "eik"
            

        cursor.updateRow(row)
        #break

# print a message indicating the update is complete
print("Field values have been reclassified using the lookup table.")
